In [24]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop
from collections import defaultdict
from sklearn.preprocessing import MinMaxScaler

In [25]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
# Data visualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
# Keras
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report

In [26]:
# Reading the data
dataset = pd.read_json('../data/renttherunway_final_data.json.gz', lines=True)
dataset = dataset.dropna()
dataset = dataset.drop(dataset[dataset['rented for']== "party: cocktail"].index)

# 0 = Small, 1 = Fit, 2 = Large
dataset.loc[dataset["fit"] == "small", "fit"] = 0

dataset.loc[dataset["fit"] == "fit", "fit"] = 1

dataset.loc[dataset["fit"] == "large", "fit"] = 2

In [27]:
# Converting the data into list of dictionaries
data = pd.DataFrame(dataset).to_dict('record')

for d in data:
    d['weight'] = int(d['weight'].split('lbs')[0])
    if(int(d['height'].split(' ')[1].split('"')[0])<10):
        height2 = int(d['height'].split(' ')[1].split('"')[0])*10
    else:
        height2 = int(d['height'].split(' ')[1].split('"')[0])
    d['height'] = int(d['height'][0][0])*100+height2
    
####converting categroies type to one hot
catogeries = ['rented for','body type']
for cat in catogeries:
    categories_list = defaultdict(int)
    for d in data:
        categories_list[d[cat]] += 1
        
    categories_id = defaultdict(int)

    i = 0
    for cID in  categories_list:
        categories_id[cID] = i
        i+=1
    for d in data:
        f = [0]*len(categories_list)
        f[categories_id[d[cat]]] = 1
        d[cat] = f[:len(categories_list)-1]

/home/nidhi/Documents/fall2021/cse258/myvirtualenv/my_project_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  


In [28]:
# Extracting features from the data
def feature(d):
    f = [1, d['rating'], len(d['review_text']), len(d['review_summary'])]
    return np.array(f)

In [29]:
# Add features
df = []
for d in data:
    arr = []
#     arr.append(d['weight'])
    arr.append(d['rating'])
    arr += d['rented for']
    arr.append(len(d['review_text']))
    arr += d['body type']
#     arr.append(d['height'])
    arr.append(d['size'])
#     arr.append(d['age'])
    df.append(arr)

In [30]:
scaler = MinMaxScaler()
scaler.fit(df)
df = scaler.transform(df)

In [31]:
y = dataset["fit"]

from keras.utils.np_utils import to_categorical
y_cat = to_categorical(y)

# Split data
test_y = y_cat[int(len(y_cat) * .8):]
train_y = y_cat[:int(len(y_cat) * .8)]
test_f = df[int(len(y_cat) * .8):]
train_f = df[:int(len(y_cat) * .8)]

# Convert to numpy array.
test_f = np.array(test_f)
train_f = np.array(train_f)

In [32]:
model = Sequential()
model.add(Dense(60, input_shape = (len(train_f[0]), ), activation = "relu"))
model.add(Dense(40, activation = "relu"))
model.add(Dropout(0.1))
model.add(Dense(3, activation = "softmax"))
model.compile(Adam(lr = 0.001), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 60)                1020      
                                                                 
 dense_7 (Dense)             (None, 40)                2440      
                                                                 
 dropout_2 (Dropout)         (None, 40)                0         
                                                                 
 dense_8 (Dense)             (None, 3)                 123       
                                                                 
Total params: 3,583
Trainable params: 3,583
Non-trainable params: 0
_________________________________________________________________


/home/nidhi/Documents/fall2021/cse258/myvirtualenv/my_project_env/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [33]:
model.fit(train_f, train_y, verbose=1, epochs=10)

Epoch 1/10
3660/3660 [==============================] - 4s 1ms/step - loss: 0.7261 - accuracy: 0.7366
Epoch 2/10
3660/3660 [==============================] - 3s 929us/step - loss: 0.7147 - accuracy: 0.7375
Epoch 3/10
3660/3660 [==============================] - 3s 948us/step - loss: 0.7119 - accuracy: 0.7381
Epoch 4/10
3660/3660 [==============================] - 3s 952us/step - loss: 0.7106 - accuracy: 0.7381
Epoch 5/10
3660/3660 [==============================] - 3s 942us/step - loss: 0.7091 - accuracy: 0.7384
Epoch 6/10
3660/3660 [==============================] - 3s 933us/step - loss: 0.7089 - accuracy: 0.7380
Epoch 7/10
3660/3660 [==============================] - 3s 954us/step - loss: 0.7080 - accuracy: 0.7384
Epoch 8/10
3660/3660 [==============================] - 3s 917us/step - loss: 0.7077 - accuracy: 0.7390
Epoch 9/10
3660/3660 [==============================] - 3s 929us/step - loss: 0.7072 - accuracy: 0.7386
Epoch 10/10
3660/3660 [==============================] - 3s 926us/

In [36]:
# Confusion matrix
# y_pred_class = model.predict_classes(test_f)
predict_x=model.predict(test_f) 
y_pred_class=np.argmax(predict_x,axis=1)

from sklearn.metrics import confusion_matrix
y_pred = model.predict(test_f)
y_test_class = np.argmax(test_y, axis=1)
confusion_matrix(y_test_class, y_pred_class)

array([[  176,  3672,    94],
       [  159, 21218,   202],
       [   72,  3453,   230]])

In [37]:
# Classification report.
from sklearn.metrics import classification_report
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       0.43      0.04      0.08      3942
           1       0.75      0.98      0.85     21579
           2       0.44      0.06      0.11      3755

    accuracy                           0.74     29276
   macro avg       0.54      0.36      0.35     29276
weighted avg       0.67      0.74      0.65     29276

